<a href="https://colab.research.google.com/github/zoya-ivanova/BigData/blob/main/Spark_Structure_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Spark Apache

### Spark Structure Streaming

In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=6d6c163f98d3c8e37ccd0459d760058767d4ae906ad05aaef23176d49739cf8c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.functions import col, sum, when, lit


### ДЗ. Условие: используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел.

In [ ]:
# Создание сессии SparkSession
spark = SparkSession.builder.master("local").getOrCreate()

# Создание потока данных (стрима)
rate_stream = spark.readStream.format("rate").load()

# Операция для подсчета суммы нечетных чисел
odd_rate_sum = rate_stream.groupBy().agg(F.sum(F.when(F.col("value") % 2 != 0,
                      F.col("value")).otherwise('next')).alias("odd_value_sum"))

# Вывести поток данных
query = odd_rate_sum.writeStream.outputMode("complete").format("console").start()

# Ждать 2 сек, завершить
query.awaitTermination(2)
query.stop()

Семинар. Задачи

1/ Условие: есть набор данных о студентах и оценках по предметам.

Используя Apache Spark, Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были струденты, в качестве столбцов были предметы, а значениями оценки.

Затем вывести средние оценки студентов по каждому предмету.

In [ ]:
spark = SparkSession.builder.appName("pivot_example").getOrCreate()

# Создаем DF
data = [("Alice", "Math", 90),
        ("Alice", "Physics", 85),
        ("Bob", "Math", 70),
        ("Bob", "Physics", 75),
        ("Bob", "History", 75),
        ("John", "Math", 65),
        ("Jonh", "Physics", 75),
        ("Jane", "Math", 100),
        ("Jane", "Physics", 90),
        ("Jane", "History", 95)]

df = spark.createDataFrame(data, ["name", "subject", "grade"])

df.show()


+-----+-------+-----+
| name|subject|grade|
+-----+-------+-----+
|Alice|   Math|   90|
|Alice|Physics|   85|
|  Bob|   Math|   70|
|  Bob|Physics|   75|
|  Bob|History|   75|
| John|   Math|   65|
| Jonh|Physics|   75|
| Jane|   Math|  100|
| Jane|Physics|   90|
| Jane|History|   95|
+-----+-------+-----+



In [ ]:

# Выполняем операцию pivot, чтобы получить таблицу с категориями в строках и днями в столбцах
df_pivot = df.groupBy("name").pivot("subject").agg(F.avg("grade"))

# Выводим результирующий DataFrame
df_pivot.show()

+-----+-------+-----+-------+
| name|History| Math|Physics|
+-----+-------+-----+-------+
| Jonh|   NULL| NULL|   75.0|
|  Bob|   75.0| 70.0|   75.0|
| John|   NULL| 65.0|   NULL|
|Alice|   NULL| 90.0|   85.0|
| Jane|   95.0|100.0|   90.0|
+-----+-------+-----+-------+



In [ ]:
# Останавливаем SparkSession
spark.stop()

2/ Найти среднее значение прожаж для каждого месяца по каждому продукту, а также для каждого продукта найти месяц с наибольшим объемом продаж

In [ ]:
spark = SparkSession.builder.appName("window_functions_example").getOrCreate()

# Создаем DF
sales_data = [(1, "2021-01-01", "product1", 100), (2, "2021-01-15", "product1", 150),
              (3, "2021-02-10", "product1", 200), (4, "2021-01-05", "product2", 120),
              (5, "2021-02-20", "product2", 180), (6, "2021-01-08", "product3", 100),
              (7, "2021-02-13", "product3", 110), (8, "2021-02-22", "product3", 220),
              (9, "2021-01-02", "product4", 225), (10, "2021-02-28", "produc4", 80),
       ]

sales_df = spark.createDataFrame(sales_data, ["id", "date", "product", "amount"])

sales_df.show()

+---+----------+--------+------+
| id|      date| product|amount|
+---+----------+--------+------+
|  1|2021-01-01|product1|   100|
|  2|2021-01-15|product1|   150|
|  3|2021-02-10|product1|   200|
|  4|2021-01-05|product2|   120|
|  5|2021-02-20|product2|   180|
|  6|2021-01-08|product3|   100|
|  7|2021-02-13|product3|   110|
|  8|2021-02-22|product3|   220|
|  9|2021-01-02|product4|   225|
| 10|2021-02-28| produc4|    80|
+---+----------+--------+------+



In [ ]:
# Оконная функция для нахождения ср знач продаж по месяцам
monthly_avg_sales_window = Window.partitionBy("product", F.month("date")).orderBy("data")

In [ ]:
# Добавление столбца со средними значениями продаж по месяцам в исходном дата фрейме

sales_df = sales_df.withColunm("monthly_avg_sales", F.avg("amount").over(monthly_avg_sales_window))


## Apache Spark Structured Streaming

1/ Создайте поток данных “rate” и просто выведите данные в консоль

In [ ]:
spark = SparkSession.builder.appName("CountElements").getOrCreate()   # Инициал Spar сессии
df = spark.readStream.format("rate").load()                           # Чтение данных из rate источника
query = df.writeStream.outputMode("append").format("console").start() # Запись в консоль
query.awaitTermination(1)                                             # Ожидание завершения записи
query.stop()

2/ Фильтрация чисел (только четные)

In [ ]:
spark = SparkSession.builder.appName("FilterEvenNumbers").getOrCreate()
df = spark.readStream.format("rate").load()
df_even = df.filter("value % 2 == 0")
query = df.writeStream.outputMode("append").format("console").start()
query.awaitTermination(2)
query.stop()

3/ Сгруппировать по значениям и посчитать количество

In [ ]:
spark = SparkSession.builder.appName("GroupByValue").getOrCreate()
df = spark.readStream.format("rate").load()
df_grouped = df.groupBy("value").count()
query = df_grouped.writeStream.outputMode("update").format("console").start()
query.awaitTermination(2)
query.stop()

4/ Вычислить сумму значений

In [ ]:
spark = SparkSession.builder.appName("SumValues").getOrCreate()
df = spark.readStream.format("rate").load()
df_sum = df.selectExpr("sum(value) AS total")
query = df_sum.writeStream.outputMode("update").format("console").start()
query.awaitTermination(2)
query.stop()

5/ Найти максимальное значение

In [ ]:
spark = SparkSession.builder.appName("MaxValue").getOrCreate()
df = spark.readStream.format("rate").load()
df_max = df.agg(F.max("value"))
query = df_max.writeStream.outputMode("update").format("console").start()
query.awaitTermination(2)
query.stop()

6/ Вычислить скользящее окно по значению

In [ ]:
spark = SparkSession.builder.appName("SlidingWindow").getOrCreate()
df = spark.readStream.format("rate").load()
df_windowed = df.groupBy(F.window("timestamp", "10 minutes")).count()
query = df_windowed.writeStream.outputMode("update").format("console").start()
query.awaitTermination(2)
query.stop()

7/ Соединение потоков данных

In [ ]:
spark = SparkSession.builder.appName("JoinStreams").getOrCreate()
df1 = spark.readStream.format("rate").load()
df2 = spark.readStream.format("rate").load()
df_joined = df1.join(df2, "value")
query = df_joined.writeStream.outputMode("append").format("console").start()
query.awaitTermination(2)
query.stop()

8/ Запись данных в файл

In [ ]:
spark = SparkSession.builder.appName("WriteToFile").getOrCreate()
df = spark.readStream.format("rate").load()
query = df.writeStream.format("parquet").option("path", "output/").start()
query.awaitTermination(2)
query.stop()